In [1]:
import numpy as np
from numba import jit

In [82]:
class HMM:
    def forward(A,B,pi,sequence):
        N=A.shape[0]
        M=B.shape[1]
        T=len(sequence)
        alpha=np.zeros([T,N])
        alpha[0]=pi*B[:,sequence[0]]
        t=1
        while True:
            if t==T:
                break
            alpha[t]=alpha[t-1]@A*B[:,sequence[t]]
            t+=1
        return(np.sum(alpha[T-1]),alpha)
    def backward(A,B,pi,sequence):
        N=A.shape[0]
        M=B.shape[1]
        T=len(sequence)
        beta=np.zeros([T,N])
        beta[T-1]=1
        t=T-2
        while True:
            if t<0:
                break
            beta[t]=A@(B[:,sequence[t+1]]*beta[t+1])
            t-=1
        return(np.sum(pi*B[:,sequence[0]]*beta[0]),beta)
    def Viterbi(A,B,pi,sequence):
        N=A.shape[0]
        M=B.shape[1]
        T=len(sequence)
        delta=np.zeros([T,N])
        psi=np.zeros([T,N])
        delta[0]=pi*B[:,sequence[0]]
        t=1
        while True:
            if t==T:
                break
            delta_A=delta[t-1,np.newaxis].T*A
            delta[t]=np.max(delta_A,axis=0)*B[:,sequence[t]]
            psi[t]=np.argmax(delta_A,axis=0)
            t+=1
        psi=psi.astype(int)
        q=np.zeros(T).astype(int)
        q[T-1]=np.argmax(delta[T-1])
        t=T-2
        while True:
            if t<0:
                break
            q[t]=psi[t+1,q[t+1]]
            t-=1
        return(q)
    def Baum_Welch(A,B,pi,sequence,max_iter):
        '''Here A, B and pi are initial values'''
        N=A.shape[0]
        M=B.shape[1]
        T=len(sequence)
        for i in range(max_iter):
            alpha=HMM.forward(A,B,pi,sequence)[1]
            beta=HMM.backward(A,B,pi,sequence)[1]
            #temporary variables
            gamma=alpha*beta/np.sum(alpha*beta,axis=1).reshape((T,1))
            #Non-vectorized version for xi
            #xi=np.zeros([N,N,T-1])
            #for t in range(T-1):
            #    xi[:,:,t]=alpha[t].reshape((N,1))*A*beta[t+1]*B[:,sequence[t+1]]
            #    xi[:,:,t]=xi[:,:,t]/np.sum(xi[:,:,t])
            xi=alpha.T[:,np.newaxis,:-1]*A[:,:,np.newaxis]*(beta*B[:,sequence].T).T[np.newaxis,:,1:]
            xi=xi/np.sum(xi,axis=(0,1))
            pi=gamma[0]
            A=np.sum(xi,axis=0)/np.sum(gamma,axis=0).reshape([N,1])
            B=np.zeros([N,M])
            for t in range(T):
                B[:,sequence[t]]+=gamma[t]
            B/np.sum(gamma,axis=0).reshape([N,1])
        return(A,B,pi)

In [83]:
M=3
N=3
pi=np.array([.3,.3,.4])
A=np.array([[.2,.3,.5],[.1,.5,.4],[.6,.1,.3]])
B=np.array([[0.1,0.5,0.4],[0.2,0.4,0.4],[0.3,0.6,0.1]])
sequence=[0,1,2,1]
print(HMM.forward(A,B,pi,sequence)[0])
print(HMM.forward(A,B,pi,sequence)[0])
print(HMM.Viterbi(A,B,pi,sequence))
HMM.Baum_Welch(A,B,pi,sequence,1)

0.01535664
0.01535664
[2 2 0 2]


(array([[ 0.28326472,  0.46607297,  0.1279398 ],
        [ 0.15195385,  0.32023844,  0.15982889],
        [ 0.38417457,  0.13276357,  0.30341065]]),
 array([[ 0.15118932,  0.56211515,  0.50711093],
        [ 0.28381208,  0.45580283,  0.3484356 ],
        [ 0.56499859,  0.98208202,  0.14445347]]),
 array([ 0.15118932,  0.28381208,  0.56499859]))